In [96]:
import pandas as pd
import numpy as np
import os
import os.path, time
import zipfile as zp
import re
import random
from datetime import datetime, date
from datetime import datetime, timedelta
from scipy import optimize
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
#import seaborn as sns; sns.set()

### RISK  - alpha, beta, gamma, RT 

In [97]:
def rp_ratio(week,sub):
    df = raw_data(week,sub)

    
    amb = df.query('data_rounds_trial_type == "ambiguity"')
    amb_d = amb[['data_rounds_reward_value', 'data_rounds_selected_sure_bet_reward']]
    amb_ratio = 1 - (round(amb_d.data_rounds_selected_sure_bet_reward.sum()/36,2))
  
    
    risk = df.query('data_rounds_trial_type == "risk"')
    risk_d = risk[['data_rounds_reward_value', 'data_rounds_selected_sure_bet_reward']]
    risk_ratio = 1 - (round(risk_d.data_rounds_selected_sure_bet_reward.sum()/34,2))
    
    return amb_ratio, risk_ratio


In [98]:
def subID_list():
    subID = ['797','806','809','826','843','845','855','856','857','1031','1153','1154','1157','1174','1262','1264','1266',
        '1364','1367','1379','1380','1381','1384','1435','1465','1467','1468','1489','1492','1493','1494','1495','1498','1519',
        '1520','1527','1532','1536','1541','1721','1722','1737','1741','1788','1791','1810','1812','1833','1934']
    return subID

In [99]:
def raw_data(week,sub):
    
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    week_label = [['01','01'],['08','11'],['17','19'],['23','26'],['32','34'],['36','42'],['44','48'],['52','56']]
    if (sub in subID99) == True:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
    else:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
        
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist_1 if subID[i] in s])
        sub_file[key].extend([s for s in filelist_2 if subID[i] in s])

    df1 = pd.read_csv(zp.ZipFile(lottery_file_1).open(sub_file[sub][0]))
    df2 = pd.read_csv(zp.ZipFile(lottery_file_2).open(sub_file[sub][1]))
    df = pd.concat([df1, df2])
    #RT_start = df['start_time']
    #RT_end = df['end_time']
    return df

def analysis_data(df):
    df.data_rounds_selected_sure_bet_reward = df.data_rounds_selected_sure_bet_reward.astype(int)
    df_temp = df[['participant_id','data_rounds_reward_probability','data_rounds_reward_value','data_rounds_selected_sure_bet_reward','data_rounds_trial_type','data_rounds_ambiguity']]
    df_temp.loc[df.data_rounds_trial_type  == 'ambiguity', 'data_rounds_reward_probability'] = 0.5
    df_temp.loc[:,'data_rounds_selected_sure_bet_reward'] = df_temp['data_rounds_selected_sure_bet_reward'].replace({0:1, 1:0})
    df_temp = df_temp.rename(columns={"data_rounds_selected_sure_bet_reward": "data_rounds_selected_variable_bet_reward"})
    x = np.array(df_temp['data_rounds_reward_value'])
    x = np.reshape(x,[70,1])
    p = np.array(df_temp['data_rounds_reward_probability'])
    p = np.reshape(p,[70,1])
    s = 5*np.ones((70,1))
    A = np.array(df['data_rounds_ambiguity'])
    A = np.reshape(A,[70,1])
    SubjectChoices = np.array(df_temp['data_rounds_selected_variable_bet_reward'])
    SubjectChoices = np.reshape(SubjectChoices,[70,1])
    exog = np.concatenate((x,p,s,A), axis=1)
    endog = SubjectChoices

    return exog, endog   

def _ll_nb2(enog, exog, alpha, beta, gamma):

    x = exog[:,0]
    p = exog[:,1]
    s = exog[:,2]
    A = exog[:,3]
    SV_X = (p-beta*(A/2)) * np.power(x,alpha)
    SV_Y = np.power(s,alpha)
    prob = 1/(1+np.exp(-gamma*(SV_X-SV_Y)))
    temp = 1-prob
    prob[enog == 0] = temp[enog == 0]
    negLL = -np.log(prob)
    return negLL

class Bin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(Bin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self,parms):
        alpha = parms[0]
        beta = parms[1]
        gamma = parms[2]
        negll = _ll_nb2(self.endog, self.exog, alpha, beta, gamma)          
        return negll

    def fit(self, start_params=None, method='L-BFGS-B',bounds = ((0, None), (-1, 1), (0, None)) ,maxiter=10000, maxfun=50000, **kwds):
        # we have one additional parameter and we need to add it for summary
        #self.exog_names.append('gamma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.array([0.9,0,1])
            # intercept
            #start_params[-2] = np.log(self.endog.mean())
        return super(Bin, self).fit(start_params=start_params,method='lbfgs',bounds = ((0, None), (-1, 1), (0, None)),
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)
    
def RT(week,sub):
    subID = subID_list()
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    
    week_label = [['01','01'],['08','11'],['17','19'],['23','26'],['32','34'],['36','42'],['44','48'],['52','56']]
    if (sub in subID99) == True:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
    else:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()

    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist_1 if subID[i] in s])
        sub_file[key].extend([s for s in filelist_2 if subID[i] in s])

    df1 = pd.read_csv(zp.ZipFile(lottery_file_1).open(sub_file[sub][0]))
    RT_start = df1['start_time'].to_list()
    RT_end = df1['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT1 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)

    df2 = pd.read_csv(zp.ZipFile(lottery_file_2).open(sub_file[sub][1]))
    RT_start = df2['start_time'].to_list()
    RT_end = df2['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT2 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)
    mean_RT = (RT1+RT2)/2
    return RT1, RT2

###  TIME - k, gamma, RT

In [100]:
def TD_ratio(week,sub):
    subID = subID_list()
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])
    
    file = sub_file[sub]
    
    if len(file) == 0:
        delay_ratio = 'NaN'
    else:
        df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
        delay = df[['data_rounds_delayed_reward', 'data_rounds_selected_immediate_reward']]
        delay_ratio = 1- delay.data_rounds_selected_immediate_reward.sum()/36
    #d3_ratio = round(delay_3.groupby(delay_3.data_rounds_delayed_reward).sum()/7,2)
    return delay_ratio

In [101]:
def TD_analysis_data(week,sub):
    subID = subID_list()
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])
    
    file = sub_file[sub]
    
    if len(file) == 0:
        exog = 'NaN'
        endog = 'NaN'
    else:
        df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
        df.data_rounds_selected_immediate_reward = df.data_rounds_selected_immediate_reward.astype(int)
        df_temp = df[['participant_id','data_rounds_delay_days','data_rounds_delayed_reward','data_rounds_selected_immediate_reward']]

        df_temp.loc[:,'data_rounds_selected_immediate_reward'] = df_temp['data_rounds_selected_immediate_reward'].replace({0:1, 1:0})
        df_temp = df_temp.rename(columns={'data_rounds_selected_immediate_reward': "data_rounds_selected_delay_reward"})
        

        SubjectChoices = np.array(df_temp['data_rounds_selected_delay_reward'])
        SubjectChoices = np.reshape(SubjectChoices,[36,1])

        v = np.array(df_temp['data_rounds_delayed_reward'])
        v = np.reshape(v,[36,1])
        d = np.array(df_temp['data_rounds_delay_days'])
        d = np.reshape(d,[36,1])
        s = 5*np.ones((36,1))

        exog = np.concatenate((v,d,s), axis=1)
        endog = SubjectChoices
    return exog, endog
         

class TD_Bin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(TD_Bin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self,parms):
        k = parms[0]
        gamma = parms[1]
        negll = TD_ll_nb2(self.endog, self.exog, k, gamma)          
        return negll

    def fit(self, start_params=None, method='lbfgs',bounds = ((0, 1), (0, None)) ,maxiter=10000, maxfun=50000, **kwds):
        # we have one additional parameter and we need to add it for summary
        #self.exog_names.append('gamma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.array([0.5,1])
            # intercept
            #start_params[-2] = np.log(self.endog.mean())
        return super(TD_Bin, self).fit(start_params=start_params,method='lbfgs',bounds = ((0, 1), (0, None)),
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)
    
def TD_ll_nb2(endog, exog, k,gamma):
    #hyperbolic function
    v = exog[:,0]
    d = exog[:,1]
    s = exog[:,2]

    SV_X = v /(1+k*d) #delay
    SV_Y = s #now
    prob = 1/(1+np.exp(-gamma*(SV_X-SV_Y)))
    temp = 1-prob
    prob[endog == 0] = temp[endog == 0]
    negLL = -np.log(prob)
    return negLL

def TD_RT(week,sub):
    subID = subID_list()
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])

    df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
    

    RT_start = df['start_time'].to_list()
    RT_end = df['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT1 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)
    RT = RT1.total_seconds()
    
    return RT

In [102]:

Week = ['1','2','3','4','5','6','7','8']
subID = subID_list()

subNo = 49

all_k = np.empty([subNo, 8])
all_TD_gamma = np.empty([subNo, 8])
all_TD_RT = np.empty([subNo,8])
all_delay_ratio = np.empty([subNo, 8])

for i in range(len(subID)):
    for t in range(len(Week)):
        exog, endog = TD_analysis_data(Week[t],subID[i])
        if exog == "NaN":
            all_k[i][t] = float('NaN')
            all_TD_gamma[i][t] = float('NaN')
            all_TD_RT[i][t] = float('NaN')
            all_delay_ratio[i][t] = float('NaN')
            
        else:
            mod = TD_Bin(endog, exog)
            res = mod.fit()
            all_k[i][t] = res.params[0]
            all_TD_gamma[i][t] = res.params[1]
            all_TD_RT[i][t] = TD_RT(Week[t],subID[i])
            all_delay_ratio[i][t] = TD_ratio(Week[t], subID[i])

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueW

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

In [103]:
np.round(all_k, 2)

array([[0.27,  nan, 0.11, 0.1 , 0.06, 0.07, 0.05, 0.04],
       [0.05,  nan, 0.06, 0.02, 0.03, 0.01, 0.02, 0.01],
       [0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.36,  nan, 0.15, 0.14, 0.2 , 0.27, 0.14, 0.26],
       [0.18,  nan, 0.27, 0.5 , 0.33, 0.26, 0.22, 0.4 ],
       [0.02, 0.04, 0.01, 0.02, 0.05, 0.01, 0.01, 0.  ],
       [0.13, 0.07, 0.06, 0.08, 0.06, 0.04, 0.06, 0.02],
       [0.12, 0.16, 0.18, 0.15, 0.24, 0.21, 0.2 , 0.23],
       [0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.17, 0.  , 0.04, 0.  , 0.05, 0.  , 0.  , 1.  ],
       [0.6 , 0.55, 0.39, 0.5 , 0.4 , 0.5 , 0.5 , 0.4 ],
       [0.07, 0.08, 0.11, 0.08, 0.07, 0.08, 0.1 , 0.07],
       [0.13, 0.21, 0.21, 0.2 , 0.03, 0.19, 0.15, 0.17],
       [0.04, 0.03, 0.05, 0.05, 0.02, 0.04, 0.06, 0.02],
       [0.04, 0.04, 0.04, 0.04, 0.02, 0.04, 0.03, 0.03],
       [0.08, 0.12, 0.06, 0.06, 0.09, 0.06, 0.06, 0.  ],
       [0.23, 0.32, 0.31, 0.33, 0.17, 0.2 , 0.32, 0.33],
       [0.03, 0.05, 0.05, 0.07,

In [122]:
np.shape(np.round(all_delay_ratio, 2))

(49, 8)

### EMOTION

In [105]:
def emotion_score(sub_id):
    positive = ['Happy', 'Enjoying myself', 'Warm/ friendly']
    negative =['4. Depressed/ blue', 'Angry / hostile', 'Criticized/put down \xa0\xa0\xa0', 'Worried/ anxious\xa0', 'Frustrated/annoyed', \
           'Hassled / pushed around']
      
    survey = ['DRM']
    subID = ['797','806','809','826','843','845','855','856','857']
    if (sub_id in subID) == True:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    
    df = pd.read_csv(subfile)

    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([sub_id])]
    time = subsetDataFrame['start_by'].unique()
    
    p_score = []
    n_score = []

    for t in range(len(time)):
        df_temp = subsetDataFrame[subsetDataFrame['start_by'].isin([time[t]])]
        q_title =  df_temp['question_title'].to_list()
    
        positive_score = 0
        negative_score = 0
        answer_title = df_temp['answer_value'].to_list()
    
        for i in range(len(q_title)):
        
        
            
            if q_title[i] in positive:
                positive_score += int(answer_title[i][0]) 
            
            if q_title[i] in negative:
                negative_score += int(answer_title[i][0])   
        
        p_score.append(positive_score)
        n_score.append(negative_score)
        
    ps = np.round(np.array(p_score)/3,2)
    ns = np.round(np.array(n_score)/6,2)
    return ps, ns

###  RISK- RT

In [106]:
subID = subID_list()
week = [1,2,3,4,5,6,7,8]

subNo = 49

all_alpha = np.empty([subNo , 8])
all_beta = np.empty([subNo , 8])
all_gamma = np.empty([subNo , 8])
all_alpha_error = np.empty([subNo ,8])
all_beta_error = np.empty([subNo ,8])
all_gamma_error = np.empty([subNo ,8])
all_RT = np.empty([subNo , 8])
all_amb_ratio = np.empty([subNo, 8])
all_risk_ratio = np.empty([subNo, 8])


for i in range(len(subID)):
    for t in range(len(week)):
        amb_ratio, risk_ratio = rp_ratio(week[t],subID[i])
        df = raw_data(week[t],subID[i])
        exog, endog = analysis_data(df)
        mod = Bin(endog, exog)
        res = mod.fit()
        all_alpha[i][t] = res.params[0]
        all_beta[i][t] = res.params[1]
        all_gamma[i][t] = res.params[2]
        all_alpha_error[i][t] = res.bse[0]
        all_beta_error[i][t] = res.bse[1]
        all_gamma_error[i][t] = res.bse[2]        
        RT1, RT2 = RT(week[t],subID[i])
        meanRT = (RT1.total_seconds()+RT2.total_seconds())/2
        all_RT[i][t] = meanRT
        all_amb_ratio[i][t] =  amb_ratio
        all_risk_ratio[i][t] = risk_ratio


//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params av

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

...

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:49

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

In [109]:
subID = subID_list()
### weekly emotion
positive_score = np.empty([subNo, 11])
negative_score = np.empty([subNo, 11])

for i in range(len(subID)):
    ps, ns = emotion_score(subID[i])
    #print(subID[i])
    if subID[i] == '1262':
        ps = np.delete(ps, [1])
        ns = np.delete(ns, [1])
    positive_score[i][:] = ps
    negative_score[i][:] = ns

#Week = [['1'],['8','11'],['17'],['23'],['32'],['36'],['44','48'],['52','56']]
#positive score 
w1 = positive_score[:,0]
w2 = np.sum(positive_score[:,1:3],axis = 1)/2
w7 = np.sum(positive_score[:,7:9],axis = 1)/2
w8 = np.sum(positive_score[:,9:11],axis = 1)/2

positive = np.empty([subNo,8])

positive[:,0] = w1
positive[:,1] = w2
positive[:,2:6] = positive_score[:,2:6]
positive[:,6] = w7
positive[:,7] = w8

#negative score
nw1 = negative_score[:,0]
nw2 = np.sum(negative_score[:,1:3],axis = 1)/2
nw7 = np.sum(negative_score[:,7:9],axis = 1)/2
nw8 = np.sum(negative_score[:,9:11],axis = 1)/2

negative = np.empty([subNo,8])

negative[:,0] = nw1
negative[:,1] = nw2
negative[:,2:6] = negative_score[:,2:6]
negative[:,6] = nw7
negative[:,7] = nw8

In [116]:
np.mean(positive, axis =1 )

array([3.06375 , 3.91625 , 4.333125, 2.6875  , 4.896875, 5.25    ,
       3.29125 , 4.62375 , 3.665625, 3.355   , 3.5     , 3.25    ,
       3.208125, 3.436875, 3.206875, 4.915625, 4.50125 , 2.313125,
       4.2275  , 4.125625, 3.8125  , 4.020625, 4.645625, 1.9575  ,
       3.271875, 3.436875, 3.125   , 1.12375 , 4.188125, 5.1225  ,
       2.      , 5.354375, 2.395   , 2.500625, 4.52125 , 1.8125  ,
       3.54125 , 3.271875, 3.2075  , 5.58375 , 5.3325  , 1.978125,
       4.374375, 4.12375 , 5.165   , 4.66625 , 3.040625, 5.043125,
       3.500625])

In [114]:
np.mean(negative, axis = 1)


array([1.823125, 1.135   , 1.40625 , 2.093125, 0.291875, 0.426875,
       0.729375, 0.      , 2.188125, 1.78    , 1.365   , 2.45875 ,
       1.501875, 2.60375 , 1.625   , 3.6975  , 0.91625 , 1.720625,
       0.926875, 0.20875 , 0.249375, 2.00125 , 0.17875 , 1.146875,
       0.98875 , 2.073125, 2.385   , 2.28    , 2.624375, 0.17875 ,
       1.084375, 0.0625  , 1.49125 , 1.57375 , 0.0625  , 1.09375 ,
       1.864375, 4.261875, 1.53125 , 0.020625, 0.646875, 0.835   ,
       1.60375 , 1.16625 , 0.78    , 0.708125, 0.956875, 0.459375,
       1.561875])

In [117]:
np.round(all_alpha,2)

array([[0.64, 0.5 , 0.56, 0.5 , 0.71, 0.52, 0.62, 0.64],
       [0.74, 1.03, 0.57, 0.53, 0.56, 0.7 , 0.57, 0.67],
       [0.63, 0.49, 0.69, 0.66, 0.73, 0.69, 0.76, 0.8 ],
       [0.56, 0.52, 0.76, 1.11, 0.71, 0.73, 0.73, 0.94],
       [1.06, 0.35, 0.5 , 0.47, 0.53, 0.79, 0.61, 0.04],
       [0.72, 0.76, 0.74, 0.42, 0.4 , 0.55, 0.48, 0.55],
       [0.81, 0.29, 0.32, 0.22, 0.28, 0.06, 0.  , 0.43],
       [0.71, 0.48, 0.26, 0.29, 0.  , 0.  , 0.  , 0.  ],
       [0.49, 0.63, 0.7 , 0.33, 0.56, 0.63, 0.66, 0.48],
       [0.37, 0.  , 0.  , 0.  , 0.64, 0.83, 0.27, 0.  ],
       [0.56, 0.42, 0.51, 0.4 , 0.51, 0.51, 0.52, 0.53],
       [0.3 , 0.62, 0.6 , 0.58, 0.58, 0.57, 0.71, 0.55],
       [0.13, 0.48, 0.87, 0.77, 0.14, 0.94, 1.08, 0.21],
       [0.47, 0.43, 0.56, 0.56, 0.45, 0.48, 0.44, 0.4 ],
       [0.6 , 0.98, 1.18, 1.6 , 1.8 , 1.35, 1.33, 1.28],
       [0.56, 0.57, 0.56, 0.55, 0.56, 0.56, 0.32, 0.06],
       [0.51, 0.54, 0.43, 0.51, 0.51, 0.5 , 0.44, 0.49],
       [1.16, 0.81, 0.13, 0.23,

In [118]:
np.round(all_amb_ratio,2)

array([[0.33, 0.14, 0.17, 0.14, 0.22, 0.11, 0.14, 0.14],
       [0.28, 0.56, 0.69, 0.56, 0.53, 0.39, 0.31, 0.31],
       [0.44, 0.19, 0.31, 0.33, 0.36, 0.33, 0.39, 0.39],
       [0.22, 0.31, 0.5 , 0.67, 0.64, 0.56, 0.58, 0.61],
       [0.39, 0.  , 0.06, 0.06, 0.11, 0.25, 0.53, 0.64],
       [0.33, 0.31, 0.25, 0.11, 0.11, 0.25, 0.31, 0.25],
       [0.33, 0.58, 0.06, 0.44, 0.64, 0.42, 0.  , 0.61],
       [0.39, 0.06, 0.06, 0.  , 0.  , 0.03, 0.03, 0.03],
       [0.44, 0.06, 0.  , 0.  , 0.03, 0.  , 0.  , 0.03],
       [0.5 , 0.81, 0.14, 0.42, 0.36, 0.44, 0.53, 0.42],
       [0.44, 0.61, 0.31, 0.08, 0.  , 0.  , 0.  , 0.  ],
       [0.56, 0.28, 0.47, 0.78, 0.67, 0.72, 0.75, 0.72],
       [0.44, 0.11, 0.39, 0.47, 0.5 , 0.47, 0.42, 0.5 ],
       [0.17, 0.22, 0.33, 0.42, 0.42, 0.5 , 0.44, 0.19],
       [0.61, 0.64, 0.72, 0.83, 0.81, 0.75, 0.75, 0.78],
       [0.78, 0.83, 0.83, 0.81, 0.83, 0.81, 0.39, 0.  ],
       [0.  , 0.56, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.5 , 0.53, 0.42, 0.47,

###  Correlation Coefficient Table

In [119]:
d={}
for x in range(len(subID)):
        corr_matx = np.empty([12,8])
        corr_matx[0][:] = all_alpha[x][:] #all_risk_ratio
        corr_matx[1][:] = all_risk_ratio[x][:]
        corr_matx[2][:] = all_beta[x][:] 
        corr_matx[3][:] = all_amb_ratio[x][:]
        corr_matx[4][:] = all_gamma[x][:]
        corr_matx[5][:] = all_RT[x][:]
        corr_matx[6][:] = all_k[x][:]
        corr_matx[7][:] = all_delay_ratio[x][:]
        corr_matx[8][:] = all_TD_gamma[x][:]
        corr_matx[9][:] = all_TD_RT[x][:]
        corr_matx[10][:] = positive[x][:]
        corr_matx[11][:] = negative[x][:]
        
        d["{0}".format(subID[x])]= corr_matx


In [120]:
subID = subID_list()
all_matx = []
c = np.zeros([12,12])
cc = c
for s in range(len(subID)):
    m = d.get(subID[s])
    corr_matx = np.empty([12,12])
    corr_p = np.empty([12,12])

    for i in range(12):
        data1 = m[i][:]
        p1 = np.argwhere(np.isnan(m[i][:]))
        for t in range(12): 
            data2 = m[t][:]
            p2 = np.argwhere(np.isnan(m[t][:]))
            p = np.unique(np.append(p1,p2))
            l = len(p)
            if l != 0:
                data1 = np.delete(m[i][:], p)
                data2 = np.delete(m[t][:], p)
                corr, _ = pearsonr(data1, data2)
            else:
                data1 = m[i][:]
                data2 = m[t][:]
                corr, _ = pearsonr(data1, data2)
            corr_matx[t][i] = corr
            #np.savetxt('../Data/Corr/corr_sub%s.txt'%subID[0], corr_matx, delimiter=',')  
    cor_matx = np.round(corr_matx,2)
    
    cor_matx[np.isnan(cor_matx)] = 0
    print ('sub is', subID[s])
    print(cor_matx)
    all_matx.append(cor_matx)
    #print(all_matx[:][:][s])
    cc += cor_matx


sub is 797
[[ 1.    0.6   0.83  0.56  0.33  0.02  0.03  0.03 -0.36  0.16  0.69 -0.16]
 [ 0.6   1.    0.38  0.26 -0.22 -0.01 -0.09  0.12 -0.35 -0.08  0.44  0.2 ]
 [ 0.83  0.38  1.    0.11  0.35 -0.4  -0.43  0.46 -0.4  -0.34  0.28  0.21]
 [ 0.56  0.26  0.11  1.    0.01  0.76  0.85 -0.81 -0.07  0.9   0.66 -0.5 ]
 [ 0.33 -0.22  0.35  0.01  1.   -0.2  -0.13  0.22  0.09 -0.12  0.18 -0.2 ]
 [ 0.02 -0.01 -0.4   0.76 -0.2   1.    0.96 -0.93 -0.    0.91  0.35 -0.45]
 [ 0.03 -0.09 -0.43  0.85 -0.13  0.96  1.   -0.99  0.07  0.96  0.28 -0.43]
 [ 0.03  0.12  0.46 -0.81  0.22 -0.93 -0.99  1.   -0.04 -0.95 -0.2   0.37]
 [-0.36 -0.35 -0.4  -0.07  0.09 -0.    0.07 -0.04  1.    0.11  0.23 -0.48]
 [ 0.16 -0.08 -0.34  0.9  -0.12  0.91  0.96 -0.95  0.11  1.    0.45 -0.62]
 [ 0.69  0.44  0.28  0.66  0.18  0.35  0.28 -0.2   0.23  0.45  1.   -0.74]
 [-0.16  0.2   0.21 -0.5  -0.2  -0.45 -0.43  0.37 -0.48 -0.62 -0.74  1.  ]]
sub is 806
[[ 1.    0.9   0.82 -0.05 -0.38  0.14 -0.06 -0.25 -0.29  0.21  0.41 -0.35]
 [

sub is 1174
[[ 1.    0.8   0.05  0.35 -0.14  0.72  0.58 -0.54 -0.42 -0.44  0.13 -0.62]
 [ 0.8   1.    0.42 -0.03 -0.31  0.77  0.57 -0.58 -0.17 -0.41 -0.08 -0.28]
 [ 0.05  0.42  1.   -0.83 -0.75 -0.03 -0.1   0.15  0.02  0.42  0.06  0.25]
 [ 0.35 -0.03 -0.83  1.    0.74  0.29  0.34 -0.38 -0.47 -0.56 -0.11 -0.23]
 [-0.14 -0.31 -0.75  0.74  1.   -0.21 -0.22  0.12 -0.16 -0.65  0.14 -0.05]
 [ 0.72  0.77 -0.03  0.29 -0.21  1.    0.93 -0.94 -0.18 -0.36 -0.49 -0.22]
 [ 0.58  0.57 -0.1   0.34 -0.22  0.93  1.   -0.98 -0.28 -0.17 -0.62 -0.02]
 [-0.54 -0.58  0.15 -0.38  0.12 -0.94 -0.98  1.    0.3   0.25  0.69 -0.03]
 [-0.42 -0.17  0.02 -0.47 -0.16 -0.18 -0.28  0.3   1.    0.02  0.16 -0.13]
 [-0.44 -0.41  0.42 -0.56 -0.65 -0.36 -0.17  0.25  0.02  1.   -0.27  0.5 ]
 [ 0.13 -0.08  0.06 -0.11  0.14 -0.49 -0.62  0.69  0.16 -0.27  1.   -0.65]
 [-0.62 -0.28  0.25 -0.23 -0.05 -0.22 -0.02 -0.03 -0.13  0.5  -0.65  1.  ]]
sub is 1262
[[ 1.    0.94  0.74  0.94 -0.79 -0.59 -0.52  0.34 -0.02 -0.27  0.22 -0.65]


sub is 1489
[[ 1.    0.89 -0.5   0.95  0.12  0.08  0.36 -0.31  0.09  0.1   0.34 -0.04]
 [ 0.89  1.   -0.56  0.91 -0.18  0.1   0.29 -0.24  0.19  0.05  0.26 -0.02]
 [-0.5  -0.56  1.   -0.73  0.22  0.16  0.4  -0.36  0.08  0.07  0.18  0.01]
 [ 0.95  0.91 -0.73  1.   -0.06  0.08  0.19 -0.18  0.02  0.12  0.28 -0.01]
 [ 0.12 -0.18  0.22 -0.06  1.   -0.07 -0.06  0.13  0.4  -0.25 -0.01  0.09]
 [ 0.08  0.1   0.16  0.08 -0.07  1.    0.73 -0.78 -0.47  0.91  0.81 -0.19]
 [ 0.36  0.29  0.4   0.19 -0.06  0.73  1.   -0.99 -0.33  0.72  0.86 -0.27]
 [-0.31 -0.24 -0.36 -0.18  0.13 -0.78 -0.99  1.    0.48 -0.8  -0.85  0.32]
 [ 0.09  0.19  0.08  0.02  0.4  -0.47 -0.33  0.48  1.   -0.74 -0.27  0.34]
 [ 0.1   0.05  0.07  0.12 -0.25  0.91  0.72 -0.8  -0.74  1.    0.77 -0.15]
 [ 0.34  0.26  0.18  0.28 -0.01  0.81  0.86 -0.85 -0.27  0.77  1.   -0.22]
 [-0.04 -0.02  0.01 -0.01  0.09 -0.19 -0.27  0.32  0.34 -0.15 -0.22  1.  ]]
sub is 1492
[[ 1.    0.78 -0.71  0.76 -0.75 -0.72  0.38 -0.33 -0.03 -0.11 -0.33 -0.04]


sub is 1737
[[ 1.   -0.7  -0.23 -0.25  0.77 -0.81 -0.59  0.59  0.59 -0.65 -0.53 -0.72]
 [-0.7   1.   -0.25  0.67 -0.96  0.69  0.38 -0.38 -0.38  0.56  0.32  0.42]
 [-0.23 -0.25  1.   -0.87  0.08  0.23  0.15 -0.15 -0.15  0.28  0.11  0.34]
 [-0.25  0.67 -0.87  1.   -0.52  0.16  0.14 -0.14 -0.14  0.07  0.09  0.01]
 [ 0.77 -0.96  0.08 -0.52  1.   -0.77 -0.57  0.57  0.57 -0.61 -0.5  -0.51]
 [-0.81  0.69  0.23  0.16 -0.77  1.    0.41 -0.41 -0.41  0.41  0.44  0.7 ]
 [-0.59  0.38  0.15  0.14 -0.57  0.41  1.   -1.   -1.    0.31  0.85  0.61]
 [ 0.59 -0.38 -0.15 -0.14  0.57 -0.41 -1.    1.    1.   -0.31 -0.85 -0.61]
 [ 0.59 -0.38 -0.15 -0.14  0.57 -0.41 -1.    1.    1.   -0.31 -0.85 -0.61]
 [-0.65  0.56  0.28  0.07 -0.61  0.41  0.31 -0.31 -0.31  1.    0.47  0.18]
 [-0.53  0.32  0.11  0.09 -0.5   0.44  0.85 -0.85 -0.85  0.47  1.    0.42]
 [-0.72  0.42  0.34  0.01 -0.51  0.7   0.61 -0.61 -0.61  0.18  0.42  1.  ]]
sub is 1741
[[ 1.    0.93  0.29  0.69 -0.75 -0.46 -0.07  0.11 -0.04 -0.56 -0.08  0.49]


In [126]:
np.round(cc/49,2)

array([[ 1.  ,  0.49,  0.18,  0.31, -0.14, -0.01, -0.04,  0.03, -0.09,
         0.02, -0.  , -0.04],
       [ 0.49,  0.98,  0.16,  0.37, -0.52,  0.07, -0.1 ,  0.09, -0.13,
         0.07, -0.  , -0.07],
       [ 0.18,  0.16,  0.98, -0.47,  0.06, -0.06, -0.03,  0.02,  0.01,
         0.09, -0.01,  0.07],
       [ 0.31,  0.37, -0.47,  1.  , -0.37, -0.02, -0.05,  0.02, -0.13,
        -0.1 , -0.01, -0.08],
       [-0.14, -0.52,  0.06, -0.37,  1.  , -0.09,  0.03, -0.03,  0.11,
        -0.08,  0.01,  0.06],
       [-0.01,  0.07, -0.06, -0.02, -0.09,  1.  ,  0.18, -0.15, -0.07,
         0.4 ,  0.13,  0.02],
       [-0.04, -0.1 , -0.03, -0.05,  0.03,  0.18,  0.94, -0.85, -0.11,
         0.16,  0.06, -0.05],
       [ 0.03,  0.09,  0.02,  0.02, -0.03, -0.15, -0.85,  0.98,  0.24,
        -0.13, -0.04,  0.03],
       [-0.09, -0.13,  0.01, -0.13,  0.11, -0.07, -0.11,  0.24,  1.  ,
        -0.04, -0.01, -0.04],
       [ 0.02,  0.07,  0.09, -0.1 , -0.08,  0.4 ,  0.16, -0.13, -0.04,
         1.  ,  0.07

In [127]:
np.round(cc/49,2)[6][:]

array([-0.04, -0.1 , -0.03, -0.05,  0.03,  0.18,  0.94, -0.85, -0.11,
        0.16,  0.06, -0.05])

In [169]:
ss = np.empty([49,12])
for i in range(49):
    ss[i][:] = all_matx[i][6]
  

In [170]:
fisherz = np.arctanh(ss)
fisherz

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in arctanh
  """Entry point for launching an IPython kernel.


array([[ 0.030009  , -0.09024419, -0.45989668,  1.25615281, -0.13073985,
         1.94591015,         inf, -2.64665241,  0.07011467,  1.94591015,
         0.28768207, -0.45989668],
       [-0.06007216, -0.06007216, -0.35409253,  0.53606034, -0.18198269,
         0.25541281,         inf, -1.58902692,  0.97295507,  0.67766607,
         0.22365611, -0.77529871],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.6041556 , -0.79281363, -0.22365611, -0.88718386, -0.08017133,
         0.97295507,         inf,        -inf,  0.86730053,  0.25541281,
         0.35409253,  0.23418947],
       [-0.82911404, -0.57633975, -0.19233717, -0.27686382, -0.1613867 ,
         0.06007216,         inf, -1.37576766,  0.88718386,  0.36544375,
        -0.19233717,  0.08017133],
       [-0.10033535,  0.1613867 ,  0.21317135, -0.35409253,  0.10033535,
         0.74141614,  

In [171]:
for i in range(12):
    np.random.seed(7654567)
    #print(stats.ttest_1samp(ss[:,i],0.0))
    print(stats.ttest_1samp(fisherz[:,i],0.0))

Ttest_1sampResult(statistic=-0.6955224272203573, pvalue=0.4900817619627277)
Ttest_1sampResult(statistic=-1.298397751990801, pvalue=0.2003544898030454)
Ttest_1sampResult(statistic=-0.7087582401626894, pvalue=0.4819027535505396)
Ttest_1sampResult(statistic=-0.7854093997072749, pvalue=0.43607335771203004)
Ttest_1sampResult(statistic=0.6824691369928529, pvalue=0.4982228603955592)
Ttest_1sampResult(statistic=3.10837944769601, pvalue=0.003159008982613833)
Ttest_1sampResult(statistic=nan, pvalue=nan)
Ttest_1sampResult(statistic=nan, pvalue=nan)
Ttest_1sampResult(statistic=nan, pvalue=nan)
Ttest_1sampResult(statistic=2.8443767105523006, pvalue=0.006521441468215335)
Ttest_1sampResult(statistic=1.1411988056071525, pvalue=0.2594488397560189)
Ttest_1sampResult(statistic=-1.1009002181383143, pvalue=0.2764303310631612)


### PLOTTING

### Positive  + Negative 

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 10))
#plt.suptitle('sub%s_choice_data\n\n gamma = %2.2f alpha = %2.2f beta = %2.2f'%(sub,res.params[2],res.params[0],res.params[1]),fontname="Helvetica", fontsize=14)

id_index = np.empty([subNo, 8])
r = np.ones((1,8))
for i in range(subNo):
    id_index[i,:] = (i+1)*np.ones((1,8))
    
    
fig, ax = plt.subplots(figsize=(20, 5))
for i in range(subNo):
    if i == 8:
        plt.scatter(id_index[i,:], positive[i,:],c = 'pink',label = 'Positive affect')
        plt.scatter(id_index[i,:], negative[i,:],c = 'powderblue',label = 'Negative affect')
    else:
        plt.scatter(id_index[i,:], positive[i,:],c = 'pink') 
        plt.scatter(id_index[i,:], negative[i,:],c = 'powderblue')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.scatter(id_index[:,1], np.mean(positive, axis = 1),s = 80,marker = 'P',c = 'crimson')
plt.scatter(id_index[:,1], np.mean(negative, axis = 1),s = 120,marker = '_',c = 'navy')
ax.legend(bbox_to_anchor=(1.05, 1))
ax.set_xlabel('Subject ')
ax.set_ylabel("Scores", fontname="Helvetica", fontsize=12)
plt.xticks([1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17], ['797','806','809','826','843','845','855','856','857','1031','1153','1154','1157','1174','1262','1264','1266'],fontname="Helvetica", fontsize=12)
plt.ylim((0,6))
#plt.show()
#plt.savefig('../Figures/emotion.png',dpi=1000)

In [ ]:
rt_m = np.mean(positive,axis = 1)
rt_std = np.std(positive,axis = 1)
#print(rt_std)
means = rt_m

#upbound = 1.96*res.bse
#lowbound = 1.96*res.bse
upbound = rt_std
lowbound = rt_std
ci = []
for i in range(len(means)):
    ci.append(((lowbound[i], upbound[i])))
# the confidence intervals you provided
#ci = [(lowbound[0], upbound[0]), (lowbound[1], upbound[1]), (lowbound[2], upbound[2])]
print(ci[:][1])
# get the range of the confidence interval
y_r = [means[i] - ci[i][1] for i in range(len(ci))]
print(means[0]-ci[0][1])
print(y_r)
fig, ax = plt.subplots()
plt.bar(range(len(means)), means, yerr=rt_std, alpha=0.2, align='center')
plt.xticks([0, 1, 2,3,4,5,6,7,8], ['797','806','809','826','843','845','855','856','857'],fontname="Helvetica", fontsize=12)
#text(x, y, s, fontsize=12)

ax.set_xlabel('Subjects')
ax.set_ylabel("Scores", fontname="Helvetica", fontsize=12)
plt.ylim((0,6))
plt.title('Positive affect',fontname="Helvetica", fontsize=12)
plt.savefig('../Figures/positive.png',dpi=1000)
plt.show()

### RT

In [ ]:
rt_m = np.mean(all_RT,axis = 1)
rt_std = np.std(all_RT,axis = 1)
#print(rt_std)
means = rt_m

# example error bar values that vary with x-position
error = rt_std
fig, ax0 = plt.subplots()
#fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(range(len(means)), means, yerr=error, fmt='-o')
ax0.set_title('Risk lottery task RT')
ax0.set_xlabel('Subjects')
ax0.set_ylabel("Second (S)", fontname="Helvetica", fontsize=12)
plt.xticks([0, 1, 2,3,4,5,6,7,8], ['797','806','809','826','843','845','855','856','857'],fontname="Helvetica", fontsize=12)
plt.ylim((0,700))
plt.title('R_RT',fontname="Helvetica", fontsize=12)
plt.savefig('../Figures/R_RT.png',dpi=1000)

In [ ]:
all_alpha

### alpha 

In [ ]:
a_m = np.mean(all_alpha,axis = 1)
a_std = np.std(all_alpha,axis = 1)

fig, ax0 = plt.subplots()
#fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(range(len(a_m)), a_m, yerr=a_std, fmt='-o')
ax0.set_xlabel('Subjects')
#ax0.set_ylabel("Second (S)", fontname="Helvetica", fontsize=12)
plt.xticks([0, 1, 2,3,4,5,6,7,8], ['797','806','809','826','843','845','855','856','857'],fontname="Helvetica", fontsize=12)
plt.ylim((0,1.5))
plt.title('Alpha',fontname="Helvetica", fontsize=12)
#plt.savefig('../Figures/Alpha.png',dpi=1000)

In [ ]:
all_beta

In [ ]:
b_m = np.mean(all_beta,axis = 1)
b_std = np.std(all_beta,axis = 1)

fig, ax0 = plt.subplots()
#fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(range(len(b_m)), b_m, yerr=b_std, fmt='-o')
ax0.set_xlabel('Subjects')
#ax0.set_ylabel("Second (S)", fontname="Helvetica", fontsize=12)
plt.xticks([0, 1, 2,3,4,5,6,7,8], ['797','806','809','826','843','845','855','856','857'],fontname="Helvetica", fontsize=12)
plt.ylim((-1,1))
plt.title('Beta',fontname="Helvetica", fontsize=12)
plt.savefig('../Figures/Beta.png',dpi=1000)

In [ ]:
all_gamma

In [ ]:
g_m = np.mean(all_gamma,axis = 1)
g_std = np.std(all_gamma,axis = 1)
print(g_m)
print(g_std)
fig, ax0 = plt.subplots()
#fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(range(len(g_m)), g_m, yerr=g_std, fmt='-o')
ax0.set_xlabel('Subjects')
#ax0.set_ylabel("Second (S)", fontname="Helvetica", fontsize=12)
plt.xticks([0, 1, 2,3,4,5,6,7,8], ['797','806','809','826','843','845','855','856','857'],fontname="Helvetica", fontsize=12)
plt.ylim((0,70))
plt.title('Gamma',fontname="Helvetica", fontsize=12)
plt.savefig('../Figures/Gamma.png',dpi=1000)